In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv(".../Updated_MS_14000.csv")
data.head()

In [ ]:
postcode = pd.read_csv("C:/Users/Mahin Vazifehdan/Desktop/Git_Learning/MSSS_on_FSscores/Patient_CAP.csv")
onset = pd.read_csv("C:/Users/Mahin Vazifehdan/Desktop/Git_Learning/MSSS_on_FSscores/Onset_date.csv")
Clinical_data_with_CAP = pd.merge(data,postcode, left_on="Patient_ID", right_on="Paziente ID", how = "left")
Clinical_data_with_CAP = pd.merge(Clinical_data_with_CAP,onset, left_on="Patient_ID", right_on="Patient_id", how = "left")
Clinical_data_with_CAP.drop(["Paziente ID","Patient_id"], axis= 1, inplace = True)
Clinical_data_with_CAP.rename(columns={"Data di nascita": "Date_of_Birth","Onset_date": "Date_of_Onset"}, inplace=True)
Clinical_data_with_CAP.drop(Clinical_data_with_CAP[Clinical_data_with_CAP["CAP"].isnull()]
                            .index, inplace = True)
Clinical_data_with_CAP['Date_of_visit'] = pd.to_datetime(Clinical_data_with_CAP['Date_of_visit'],
                                                         format ='%Y-%m-%d')
Clinical_data_with_CAP['Date_of_Onset'] = pd.to_datetime(Clinical_data_with_CAP['Date_of_Onset'],
                                                         format ='%Y-%m-%d')
Clinical_data_with_CAP["CAP"] = Clinical_data_with_CAP["CAP"].astype(int)

first = Clinical_data_with_CAP.groupby('Patient_ID').head(1)
last = Clinical_data_with_CAP.groupby('Patient_ID').tail(1)
result = pd.concat([first,last]).drop_duplicates().sort_values(by=['Patient_ID','Date_of_visit'])

result = result[['Patient_ID','CAP','Date_of_Birth','Date_of_Onset','Date_of_visit']]
result.head()

In [ ]:
df_grouped = result.groupby('Patient_ID')['Date_of_visit'].agg(First_Date_of_visit='min',
                                                               Last_Date_of_visit='max').reset_index()
df_grouped.head()

In [ ]:
Clinical_data_with_CAP = pd.merge(result,df_grouped, left_on="Patient_ID", right_on="Patient_ID", how = "left")
result = Clinical_data_with_CAP.groupby('Patient_ID').head(1).reset_index()
result[['Patient_ID','CAP','Date_of_Birth','Date_of_Onset','First_Date_of_visit','Last_Date_of_visit']].to_csv("First_Last_Patient.csv", index = False)